 
# Federated Learning Training Plan: Execute Plan

Here we load and execute Plan and Model params created earlier in "Create Plan" notebook. 

This represents PySyft (python) worker.

In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")

import syft as sy
import torch as th
from torchvision import datasets, transforms
from syft.serde import protobuf
from syft_proto.execution.v1.plan_pb2 import Plan as PlanPB
from syft_proto.execution.v1.state_pb2 import State as StatePB
from syft import PlaceHolder
from syft.execution.state import State
import os
import numpy as np

sy.make_hook(globals())
# force protobuf serialization for tensors
hook.local_worker.framework = None

W0527 13:48:11.935419 18544 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'D:\Anaconda3\envs\syft\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0527 13:48:11.996123 18544 deprecation_wrapper.py:119] From D:\Anaconda3\envs\syft\lib\site-packages\tf_encrypted\session.py:24: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



Setting up Sandbox...
Done!


Utility func that unserializes file contents into PySyft classes.
Note that we must know file contents beforehand to use specific protobuf class for deserialization.

In [2]:
def deserializeFromBin(worker, filename, pb):
    with open(filename, "rb") as f:
        bin = f.read()
    pb.ParseFromString(bin)
    return protobuf.serde._unbufferize(worker, pb)

def serializeToBinPb(worker, obj, filename):
    pb = protobuf.serde._bufferize(worker, obj)
    bin = pb.SerializeToString()
    print("Writing %s to %s/%s" % (obj.__class__.__name__, os.getcwd(), filename))
    with open(filename, "wb") as f:
        f.write(bin)

## Step 4: Unserialize Plan & Model

In [3]:
training_plan = deserializeFromBin(hook.local_worker, "tp_full.pb", PlanPB())
model_params_state = deserializeFromBin(hook.local_worker, "model_params.pb", StatePB())
# unwrap tensors from State
model_params = model_params_state.tensors()

print("Loaded syft plan:", training_plan.code)
print("Loaded torchscript plan code:", training_plan.torchscript.code)
print("Loaded params count:", len(model_params))
print("Params shapes:", [p.shape for p in model_params])

Loaded syft plan: def training_plan(arg_1, arg_2, arg_3, arg_4, arg_5, arg_6, arg_7, arg_8):
    2 = arg_1.dim()
    var_0 = arg_5.t()
    var_1 = arg_1.matmul(var_0)
    var_2 = arg_6.add(var_1)
    var_3 = var_2.relu()
    2 = var_3.dim()
    var_4 = arg_7.t()
    var_5 = var_3.matmul(var_4)
    var_6 = arg_8.add(var_5)
    var_7 = var_6.max()
    var_8 = var_6.sub(var_7)
    var_9 = var_8.exp()
    var_10 = var_9.sum(dim=1, keepdim=True)
    var_11 = var_10.log()
    var_12 = var_8.sub(var_11)
    var_13 = arg_2.mul(var_12)
    var_14 = var_13.sum()
    var_15 = var_14.neg()
    out_1 = var_15.div(arg_3)
    var_16 = out_1.mul(0)
    var_17 = var_16.add(1)
    var_18 = var_17.div(arg_3)
    var_19 = var_18.mul(-1)
    var_20 = var_13.mul(0)
    var_21 = var_20.add(1)
    var_22 = var_21.mul(var_19)
    var_23 = var_22.mul(var_12)
    var_24 = var_22.mul(arg_2)
    var_25 = var_23.copy()
    var_26 = var_24.add(0)
    var_27 = var_24.mul(-1)
    var_28 = var_27.sum(dim=[1], keepdim=T

## Step 5: Train!

Define the full training procedure that uses Plan as one training step on a batch of data. 

In [4]:
batch_size = 64
mnist = th.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor()),
    batch_size=batch_size,
    shuffle=True
)

def execute_training_plan(data, plan, model_params, epochs=1, batch_size=th.tensor(batch_size), lr=th.tensor(0.001)):
    for epoch in range(1, epochs+1):
        losses = []
        accuracies = []
        for batch_idx, (X, y) in enumerate(data):
            X = X.view(X.shape[0], -1)
            y_oh = th.nn.functional.one_hot(y, 10)
            loss, acc, *model_params = plan(X, y_oh, batch_size, lr, model_params)
            losses.append(loss.item())
            accuracies.append(acc.item())
            if batch_idx % 100 == 0:
                print("Batch %d, loss: %f, accuracy: %f" % (batch_idx, loss, acc), end="\r")
        print('Epoch %d, avg loss: %f, avg training accuracy: %f' % (epoch, np.mean(losses), np.mean(accuracies)))
    return model_params

To show both variants of plans work, first we run training with "list of ops" Plan and get updated model weights,
then execute training with torchscript Plan starting with updated weights and get further updated weights :)

In [5]:
# Plain Plan
training_plan.validate_input_types = False
updated_model_params = execute_training_plan(mnist, training_plan, model_params)

Epoch 1, avg loss: 2.194609, avg training accuracy: 0.431253


In [6]:
# Torchscript Plan
# NOTE that execution point is `.torchscript` property
updated_model_params = execute_training_plan(mnist, training_plan.torchscript, updated_model_params)

Epoch 1, avg loss: 1.956141, avg training accuracy: 0.675856


## Step 6: Create Diff

Naive diff is just a difference between original model weights and updated model weights. 

In [7]:
diff = [ model_params[i] - updated_model_params[i] for i in range(len(model_params)) ]

print([ item.shape for item in diff ])

[torch.Size([392, 784]), torch.Size([392]), torch.Size([10, 392]), torch.Size([10])]


Let's wrap it in State to serialize.

In [8]:
diff_state = State(
    state_placeholders=[PlaceHolder().instantiate(param) for param in diff]
)
serializeToBinPb(hook.local_worker, diff_state, 'diff.pb')




Writing State to D:\projects\openmined\PySyft\examples\experimental\FL Training Plan/diff.pb
